In [50]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_cohere import CohereRerank


In [51]:
import os
os.environ["GROQ_API_KEY"] = ""

embedding_model = OllamaEmbeddings(model="nomic-embed-text")

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.1,
    max_retries=2,
)

In [52]:
#Carregar pdf

pdf_link = "lotus.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [61]:
#Separar em chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 4000,
    chunk_overlap = 20,
    length_function = len,
    add_start_index = True
)

chunks = text_splitter.split_documents(pages)
print(chunks)

[Document(metadata={'source': 'lotus.pdf', 'page': 0, 'page_label': '1', 'start_index': 0}, page_content='Pavan Emani\nSummary\nThe article discusses the transition from Text2SQL and Retrieval-Augmented\nGeneration (RAG) to Table-Augmented Generation (TAG) as a more effective\nmethod for AI-driven data queries, leveraging the LOTUS framework to integrate\nAI and database capabilities.\nAbstract\nThe article "Goodbye, Text2SQL: Why Table-Augmented Generation (TAG) is\nthe Future of AI-Driven Data Queries!" explores the limitations of current AI\nmethods like Text2SQL and Retrieval-Augmented Generation (RAG) in handling\ncomplex data queries. It argues that TAG, a new approach developed by\nresearchers from Stanford and Berkeley, overcomes these limitations by\ncombining AI\'s semantic reasoning with the computational power of databases.\nTAG utilizes a multi-step process involving query synthesis, execution, and\nanswer generation, enhanced by the LOTUS framework, which allows for\nsema

In [ ]:
# Storages
db = Chroma.from_documents(chunks, embedding=embedding_model, persist_directory="naiveDB2")

vector_db = Chroma(persist_directory="naiveDB2", embedding_function=embedding_model)

In [65]:
naive_retreiver = vector_db.as_retriever(search_kwargs={"k":10})
print(vector_db.get())

os.environ["COHERE_API_KEY"] = ""

rerank = CohereRerank(top_n=3, model="rerank-v3.5")

compressor_retriever = ContextualCompressionRetriever(
    base_compressor=rerank,
    base_retriever=naive_retreiver
)

{'ids': ['ea8e33f3-0766-401e-8ec7-5887baa85fa3', 'b74267ca-72ab-4f3e-a45c-d90d58cd7bbb', '143bc5e3-20c2-428e-b3ec-3dc46c2f96ba', '74ff5533-087b-431d-9ed8-e07d63aadead', 'b178aff5-be43-4c9e-af6e-3f296cfd6ecd', 'f5a96d0f-5abb-4378-833b-f3e0ff85a7c7', '62f924e7-1ab6-4ad8-8021-7b02dc016ab6', 'bbdb3a71-22af-498c-ab8d-311d196d8a0d', 'cbb55b68-1f6e-421c-ac5c-722f7ec33986'], 'embeddings': None, 'documents': ['Pavan Emani\nSummary\nThe article discusses the transition from Text2SQL and Retrieval-Augmented\nGeneration (RAG) to Table-Augmented Generation (TAG) as a more effective\nmethod for AI-driven data queries, leveraging the LOTUS framework to integrate\nAI and database capabilities.\nAbstract\nThe article "Goodbye, Text2SQL: Why Table-Augmented Generation (TAG) is\nthe Future of AI-Driven Data Queries!" explores the limitations of current AI\nmethods like Text2SQL and Retrieval-Augmented Generation (RAG) in handling\ncomplex data queries. It argues that TAG, a new approach developed by\nres

In [66]:
TEMPLATE = """
    Você é um especialista em inteligência artificial. Responda a pergunta abaixo utilizando o contexto informado.
    Query:
    {question}

    Context:
    {context}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)


In [67]:
setup_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": compressor_retriever})

output_parser = StrOutputParser()

In [68]:
compressor_chain_retriever = setup_retrieval | rag_prompt | llm | output_parser

In [69]:
compressor_chain_retriever.invoke("Faça um resumo do contexto?")

Number of requested results 10 is greater than number of elements in index 9, updating n_results = 9


'Aqui está um resumo do contexto:\n\nO contexto discute a limitação dos métodos atuais de inteligência artificial (IA) para consultar bancos de dados, como Text2SQL e Retrieval-Augmented Generation (RAG). Esses métodos têm dificuldade em lidar com perguntas complexas que requerem raciocínio semântico e conhecimento não explícito nos bancos de dados.\n\nPara superar essas limitações, é apresentado o framework LOTUS (Leveraging Optimization Techniques for Unifying Semantic Queries), que integra modelos de linguagem grande (LLMs) diretamente no pipeline de processamento de consultas de banco de dados. Isso permite que as consultas sejam realizadas de forma mais complexa e significativa, combinando a capacidade de raciocínio dos LLMs com a força computacional dos bancos de dados.\n\nO framework LOTUS é utilizado pelo sistema TAG (Table Augmentation Generation), que consiste em três etapas:\n\n1. Síntese de consultas: a pergunta do usuário é traduzida em uma consulta SQL.\n2. Execução de co